<a href="https://colab.research.google.com/github/henrigeuer/stratosphaerenflug/blob/main/strato_auswerung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datenauswerung mit Python
Um zu beginnen, die einzelnen Codeabschnitte in ihrer Reihenfolge ausführen.



##Importieren der benötigten Bibliotheken
Die Bibliotheken numpy und mathplotlib dollrn verwendet werden, um die Daten darzustellen

In [5]:
import numpy as np
import matplotlib.pyplot as plt

##Laden der Messwerte aus Github
Bevor wir mit den Daten arbeiten können, müssen wir sie zunächst erstmal laden.
Die Rohdaten befinden sich ebenfalls im Github Reposotory. Von dort werden sie nun, zunächst als String, geladen.

In [19]:
#Laden des Github Repositories
!git clone https://github.com/henrigeuer/stratosphaerenflug.git

fatal: destination path 'stratosphaerenflug' already exists and is not an empty directory.


In [99]:
#Speichern der Dateipfade
path_arduino = "/content/stratosphaerenflug/LOG.TXT"
path_strato = "/content/stratosphaerenflug/F_24.LOG"

#Öffnen der Dateien
file_arduino = open(path_arduino)
file_strato = open(path_strato)

#Zwischenpeichern der Daten
data_arduino = file_arduino.read()
data_strato = file_strato.read()

#Schließen der Dateien
file_arduino.close()
file_strato.close()

## Vorbereiten der Messwerte
Um die Daten grafisch darstellen zu können, müssen wir sie nun weiter Verarbeiten. Dabei sollen alle gemessenen Daten in einzelne Listen gespeichert werden 

In [93]:
#Daten des Arduino
#Nach Zeilen trennen
data_arduino_rows = data_arduino.split("\n")
#Leere Zeilen entfernen
while "" in data_arduino_rows:
  data_arduino_rows.remove("")
#erste Zeile entfernen
data_arduino_rows.pop(0)

#Aufteilen jeder Zeile in einzelne Messwerte
data_arduino_elements =[]
for row in data_arduino_rows:
  data_arduino_elements.append(row.split(";"))

#Erstellen von Messwertspezifischen Listen
ino_temp = []
ino_hum = []
ino_press =[]
ino_tvoc= []
ino_co2 = []
ino_uv = []
for row in data_arduino_elements:
  ino_temp.append(float(row[0]))
  ino_hum.append(float(row[1]))
  ino_press.append(float(row[2]))
  ino_tvoc.append(float(row[3]))
  ino_co2.append(float(row[4]))
  ino_uv.append(float(row[5]))

#Zeitliste im Intervall 5sek erzeugen
time = []
i=0
while len(time)<len(ino_temp):
  time.append(i)
  i+=5



In [110]:
#Daten des Strato3
#Nach Zeilen trennen
data_strato_rows = data_strato.split("\n")
#Leere Zeilen entfernen
while "" in data_strato_rows:
  data_strato_rows.remove("")

#erste zwei Zeile entfernen
data_strato_rows.pop(0)
data_strato_rows.pop(0)

#Aufteilen jeder Zeile in einzelne Messwerte
data_strato_elements =[]
for row in data_strato_rows:
  data_strato_elements.append(row.split(";"))
# 'Up-Time', 'UTC', 'Date', 'RMC Valid', 'Sats in use', 'Latitude', 'Longitude', 'Speed over Ground [knots]', 'Speed over Ground [km/h]', 'Course over Ground', 'Altitude NN [m]', 'Board: Temp [C]', 'Extern: Temp [C]', 'Extern: Hum [perc]', 'Extern: Press [hPa]', 'Batt Voltage [V]', 'Logger Status'

#nicht Nutzbare Werte(Up-Time, UTC,Date, RMC Valid, Sats in use, Latitude, Longitude, Logger Status) entfernen
for element in data_strato_elements:
  del element[0:8]
  element.pop(-1)


[['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '69.695', '1021.130', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '69.634', '1021.109', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '69.534', '1021.169', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '17.0', '69.420', '1021.109', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '17.0', '69.351', '1021.150', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '69.298', '1021.90', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '17.125', '69.244', '1021.70', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '16.875', '69.199', '1021.80', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '16.750', '69.77', '1021.99', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '68.939', '1021.130', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.750', '16.875', '68.840', '1021.70', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '17.0', '68.764', '1021.0', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '16.875', '68.710', '1021.109', '9.2'], ['NA', 'NA', 'NA', 'NA', '19.875', '16

In [94]:
def plot_xy(y, x= time):
  plt.xlim(min(x), max(x))
  plt.ylim(min(y)-5, max(y)+5)
  plt.plot(x, y)

